In [ ]:
import pandas as pd
import os
import seaborn as sns
import shutil

os.environ["CRYPTOGRAPHY_OPENSSL_NO_LEGACY"] = "yes"

# ? Se usa para generar gráficos
import matplotlib.pyplot as plt

# ? Se usa para determina la fecha de ejecución del proceso dentro del export de los resultados estadísticos
from datetime import datetime

# ? Conexión a BD Postgres. Se usa dado que pandas no acepta sino este paquete para la conexión.
from sqlalchemy import create_engine, text

from datetime  import datetime as dt

from arcgis.features import GeoAccessor, GeoSeriesAccessor
import arcpy

from pathlib import Path

# ** Librerías propias
import funcion_dataframe_terrenos

In [ ]:
RUTA_PREDIOS_GLOBALES = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\zApoyo_A_Equipos\zApoyoEquipoJuridico\9.Analisis_Predios_LimiteAdministrativo\zInsumos\predios_en_limite_administrativo.xlsx"
NOMBRE_HOJA = 'Predios En Límite Administrativ'

DIRECTORIO_SALIDA_CSV_SEGUIMIENTO = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\zApoyo_A_Equipos\zApoyoEquipoJuridico\9.Analisis_Predios_LimiteAdministrativo\zInsumos"
NOMBRE_CSV_SEGUIMIENTO = 'Seguimiento_Avance_Verificacion_Predios_Vs_LimiteAdministrativo.csv'

tipo_campos_predios_globales = {'NUMERO_PREDIAL':str,
    'CODIGO_TERRENO':str,
    'CODIGO_MUNICIPIO':str,
    'MATRICULA_INMOBILIARIA':str,
    'TERRENO':str,
    'CODIGO_MAN_VEREDA':str,
    'NUMERO_DOCUMENTO':str,
    'Area_ha_cmt12':float}

In [ ]:
df_predGlobalesSobreLimite = pd.read_excel(RUTA_PREDIOS_GLOBALES, sheet_name=NOMBRE_HOJA, dtype=tipo_campos_predios_globales)
df_predGlobalesSobreLimite['Analizado'] = None
df_predGlobalesSobreLimite

In [ ]:
sheet_name = 'Predio_LimiteAdministrativo_MariaLaBaja'
sheet_id = '142IZnazQhFzVKgMGvozYSMnFGqoU_QwBVx1yWavGZqA'


url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

df_prediosAnalizadosLimites = pd.read_csv(url)
df_prediosAnalizadosLimites = pd.DataFrame(df_prediosAnalizadosLimites['CODIGO'])

In [ ]:
df_cruce_total_vs_analizado = pd.merge(
    left=df_predGlobalesSobreLimite,
    right=df_prediosAnalizadosLimites,
    left_on='CODIGO_TERRENO',
    right_on='CODIGO',
    how='left'
    )

In [ ]:
df_cruce_total_vs_analizado.loc[df_cruce_total_vs_analizado['CODIGO'].notnull(),'Analizado'] = 'SI'
df_cruce_total_vs_analizado.loc[df_cruce_total_vs_analizado['CODIGO'].notnull(),'Analizado'] = 'SI'

df_cruce_total_vs_analizado['Area_ha_cmt12'] = df_cruce_total_vs_analizado['Area_ha_cmt12'].round(3)

In [ ]:

ruta_salida_csv = os.path.join(DIRECTORIO_SALIDA_CSV_SEGUIMIENTO, NOMBRE_CSV_SEGUIMIENTO)
df_cruce_total_vs_analizado.to_csv(ruta_salida_csv)
print(f"Se crea CSV Seguimiento {NOMBRE_CSV_SEGUIMIENTO}")